In [1]:
# Environment setup and module import
import torch
from torch.utils import data
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
debug_encoding = True # if True, uses existing mini_train_encoded.csv file so this runs fast
save_data = True # default: False. This is for diagnostic purposes.  Remove later.

In [3]:
class Dataset(data.Dataset):
  def __init__(self, list_IDs, labels):
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        return len(self.list_IDs)

  def __getitem__(self, index):
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        x = torch.load('data/' + ID + '.pt')
        y = self.labels[ID]

        return x, y

In [4]:
params = {'batch_size': 10,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 100

In [5]:
# Import DF from CSV

if debug_encoding == True:
    dataset = 'data/mini_train_encoded.csv'
else:
    dataset = 'data/train_encoded.csv'

df = pd.read_csv(dataset)


In [6]:
# Remove hasdetections from the df.  Make it a new df for labels.
label_df = pd.DataFrame()
label_df = df['HasDetections'].to_numpy()
df.drop(columns=['HasDetections'])


# Flatten all other variables into a single element
train_df = pd.DataFrame()
df['concat'] = pd.Series(df.fillna('').values.tolist()).map(lambda x: ''.join(map(str,x)))
df['concat'] = df['concat'].str.replace('.','. ')
df['concat'] = df['concat'].str.replace('-','')

train_df = df['concat'].to_numpy()

print(label_df.shape)
print(train_df.shape)


(312251,)
(312251,)


In [7]:
# Generators
training_set = Dataset(train_df, label_df)
training_generator = data.DataLoader(training_set, **params)

#validation_set = Dataset(partition['validation'], labels)
#validation_generator = data.DataLoader(validation_set, **params)

# Loop over epochs
#for epoch in range(max_epochs):
    # Training
#    for local_batch, local_labels in training_generator:
        # Transfer to GPU
#        local_batch, local_labels = local_batch.to(device), local_labels.to(device)

        # Model computations
#        [...]

    # Validation
#    with torch.set_grad_enabled(False):
#        for local_batch, local_labels in validation_generator:
#            # Transfer to GPU
#            local_batch, local_labels = local_batch.to(device), local_labels.to(device)

#            # Model computations
#            [...]

In [8]:
if save_data == True:
    pd.DataFrame(label_df).to_csv("data/pt-temp-label.csv")
    pd.DataFrame(train_df).to_csv("data/pt-temp-train.csv")